In [7]:
import pandas as pd
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch

# Load the dataset
df = pd.read_csv('dataset.csv')

from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights

# Load the EfficientNetV2-S model
weights = EfficientNet_V2_S_Weights.DEFAULT
model = efficientnet_v2_s(weights=weights)

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

# Custom dataset
class ClothingDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        img_path = f"images/{img_name}.png"
        image = Image.open(img_path).convert("RGB")
        labels = self.dataframe.iloc[idx, 1:].values.astype(np.float32)
        if self.transform:
            image = self.transform(image)
        return image, labels

# Create dataset instances
dataset = ClothingDataset(df, transform=preprocess)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [8]:
import torch
import torch.nn as nn

# Modify output layer
num_clothing_attributes = 66  # Number of attributes in your dataset
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2),  # Optional dropout for regularization
    torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=num_clothing_attributes)
)

for param in model.features.parameters():
    param.requires_grad = False


# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cuda


In [10]:
import torch.optim as optim
import torch.nn as nn

# Loss Function: Binary Cross Entropy with Logits Loss is suitable for multi-label tasks
criterion = nn.BCEWithLogitsLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Additional: Learning Rate Scheduler (Optional but recommended)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Validation Function
def validate(model, val_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():  # No gradients are needed for validation
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            

    val_loss /= len(val_loader)
    return val_loss

# Training Loop
num_epochs = 10  # Set number of epochs
best_val_loss = float('inf')  # Keep track of the best validation loss
patience = 3  # Number of epochs to wait for improvement
epochs_without_improvement = 0
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):  # Iterate through the training data
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device

        optimizer.zero_grad()   # Zero the parameter gradients

        outputs = model(inputs) # Forward pass
        loss = criterion(outputs, labels)   # Calculate loss
        loss.backward()         # Backpropagation
        optimizer.step()        # Update weights

        running_loss += loss.item()  # Accumulate loss

    # After each epoch, print the average loss and adjust learning rate (if using a scheduler)
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}') 
    
    # Validation after each epoch
    val_loss = validate(model, val_loader, criterion)  # Pass device to validate function
    print(f'Epoch {epoch + 1}/{num_epochs}, Val Loss: {val_loss:.4f}')
    
    # Early Stopping Logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        epochs_without_improvement = 0  # Reset the counter
    else:
        epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f'Early stopping triggered after {epoch+1} epochs.')
            break  # Exit the training loop
    
    # Save the model if the validation loss is the best so far
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')  # Save model state

    # If using a learning rate scheduler, step it after each epoch
    scheduler.step()  


C:\Users\jasper\PycharmProjects\preloved-auto-tag\.venv\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10, Loss: 0.1882
Epoch 1/10, Val Loss: 0.2021


KeyboardInterrupt: 

In [ ]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Evaluation phase
model.eval()
val_loss = 0.0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item() * images.size(0)

val_loss /= len(val_loader.dataset)
print(f'Validation Loss: {val_loss:.4f}')
